In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from autogluon.tabular import TabularPredictor

In [36]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 6.6 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━

In [4]:
train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')
sample = pd.read_csv('/kaggle/input/playground-series-s5e6/sample_submission.csv')

In [5]:
X = train.drop(['Fertilizer Name'],axis = 1)
y = train['Fertilizer Name']

In [6]:
# Standardize 'Soil Type' in both datasets
X['Soil Type'] = X['Soil Type'].str.strip().str.lower()
test['Soil Type'] = test['Soil Type'].str.strip().str.lower()

In [7]:
from sklearn.preprocessing import LabelEncoder

# Fit the encoder on training data
label_encoder = LabelEncoder()
X['Soil Type'] = label_encoder.fit_transform(X['Soil Type'])

# Transform the test data
test['Soil Type'] = label_encoder.transform(test['Soil Type'])

In [4]:
train.isna().sum()

id                 0
Temparature        0
Humidity           0
Moisture           0
Soil Type          0
Crop Type          0
Nitrogen           0
Potassium          0
Phosphorous        0
Fertilizer Name    0
dtype: int64

In [6]:
train['Fertilizer Name'].value_counts()

Fertilizer Name
14-35-14    114436
10-26-26    113887
17-17-17    112453
28-28       111158
20-20       110889
DAP          94860
Urea         92317
Name: count, dtype: int64

In [8]:
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
label_encoder = LabelEncoder()
for column in categorical_cols:
    X[column] = label_encoder.fit_transform(X[column])
categorical_cols_test = test.select_dtypes(include=['object', 'category']).columns.tolist()
for column in categorical_cols_test:
    test[column] = label_encoder.transform(test[column])

In [9]:
X['Fertilizer Name'] = y

In [10]:
X.head(3)

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,37,70,36,1,8,36,4,5,28-28
1,1,27,69,65,4,4,30,6,18,28-28
2,2,29,63,32,4,4,24,12,16,17-17-17


In [18]:
predictor = TabularPredictor(label='Fertilizer Name',
            eval_metric='accuracy').fit(train_data=X,
            presets= 'best_quality',
            time_limit=1200)# 600)

No path specified. Models will be saved in: "AutogluonModels/ag-20250601_190204"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       27.70 GB / 31.35 GB (88.4%)
Disk Space Avail:   19.06 GB / 19.52 GB (97.6%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be 

In [19]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                    model  score_val eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0     WeightedEnsemble_L3   0.176869    accuracy      81.687197  761.299343                0.081940          16.772377            3       True          9
1  NeuralNetFastAI_BAG_L2   0.176444    accuracy      81.605258  744.526965               10.198827         191.489967            2       True          7
2       LightGBMXT_BAG_L1   0.175451    accuracy      59.520736  151.941327               59.520736         151.941327            1       True          4
3     WeightedEnsemble_L2   0.175451    accuracy      59.601098  163.999842                0.080363          12.058515            2       True          6
4       LightGBMXT_BAG_L2   0.173991    accuracy      77.391852  611.293467                5.985422          58.256469            2       True          8
5  NeuralNetFa

{'model_types': {'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel',
  'NeuralNetFastAI_BAG_L2': 'StackerEnsembleModel_NNFastAiTabular',
  'LightGBMXT_BAG_L2': 'StackerEnsembleModel_LGB',
  'WeightedEnsemble_L3': 'WeightedEnsembleModel'},
 'model_performance': {'KNeighborsUnif_BAG_L1': 0.14673333333333333,
  'KNeighborsDist_BAG_L1': 0.14467066666666667,
  'NeuralNetFastAI_BAG_L1': 0.16136533333333333,
  'LightGBMXT_BAG_L1': 0.17545066666666667,
  'LightGBM_BAG_L1': 0.15750666666666666,
  'WeightedEnsemble_L2': 0.17545066666666667,
  'NeuralNetFastAI_BAG_L2': 0.176444,
  'LightGBMXT_BAG_L2': 0.17399066666666665,
  'WeightedEnsemble_L3': 0.17686933333333332},
 'model_best': 'WeightedEnsemble_L3',
 'model_paths': {'KNei

In [20]:
predictions = predictor.predict(test)#.evaluate(test)
predictions.head()

0         DAP
1    17-17-17
2    10-26-26
3    14-35-14
4       20-20
Name: Fertilizer Name, dtype: object

In [15]:
sample.head()

,id,Fertilizer Name
0,750000,14-35-14 10-26-26 Urea
1,750001,14-35-14 10-26-26 Urea
2,750002,14-35-14 10-26-26 Urea
3,750003,14-35-14 10-26-26 Urea
4,750004,14-35-14 10-26-26 Urea


In [21]:
sample["Fertilizer Name"] = predictions 
sample.to_csv("submission.csv", index=False)

In [22]:
sample.head()

,id,Fertilizer Name
0,750000,DAP
1,750001,17-17-17
2,750002,10-26-26
3,750003,14-35-14
4,750004,20-20
